In [ ]:
!pip install -q transformers datasets numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("drive/Shareddrives/BPM PROJECT/Dataset/5_products_dataset_preprocessed.csv")
df

,product_id,review_body,review_headline,star_rating,helpful_votes,total_votes,review_date,review_year
0,B003L1ZYYM,job,Five Stars,5,0,0,2015-08-31,2015
1,B0001FTVEK,great product,Five Stars,5,0,0,2015-08-31,2015
2,B0012S4APK,great product fair price easy install worked f...,Great product at a fair price,5,0,0,2015-08-31,2015
3,B003EM8008,probably best earphones ever better apple ones...,Good quality head phones!,5,0,0,2015-08-31,2015
4,B0001FTVEK,work great sounds amazing,Four Stars,4,0,0,2015-08-31,2015
...,...,...,...,...,...,...,...,...
54385,B0001FTVEK,headphones months ca n't believe life changed ...,Can't understand the negative reviews!,4,134,148,2004-07-31,2004
54386,B0002L5R78,tried quite cables past months hd setup far be...,Best HDMI Cable,5,10,18,2004-07-12,2004
54387,B0001FTVEK,first set wireless headphones 've looking set ...,"OK headphones, but not what I was looking for...",3,27,30,2004-07-07,2004
54388,B0001FTVEK,lived europe 2003 purchased pair sennheiser wi...,Extremely disappointed,1,311,360,2004-07-07,2004


In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation
  #creo un array di testi concatenati, ognuno dei quali ha massimo 1024 token
  #lo stesso procedimento verrà fatto con i riassunti generati!
  # si parte da checkpoint in poi, fino a finire il ciclo.

  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      #print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    #print("Number of reviews processed: {}".format(i))

  return batches

In [ ]:
def summarize_final(df):
  ids = df['product_id'].unique()
  years = df['review_year'].unique()
  for id in ids:
    for year in years:
      part = df[(df["product_id"] == id) & (df["review_year"] == year)]
      reviews = part["review_body"].tolist()
      batches = get_batches(reviews, False)

      summaries_first_stage = []
      for input in batches:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        # print(decoded_output)
        summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      input = "summarize: "
      count = 0
      checkpoint = 0
      batches_second_stage = []
      i = 0
      for i in range(0,len(summaries_first_stage)):
        if count + len(summaries_first_stage[i].split(" ")) < 1024:
          #print(cleaned_reviews[i])
          input = input + summaries_first_stage[i] + " . "
          #print(input)
          count = count + len(summaries_first_stage[i].split(" "))
        else:
          batches_second_stage.append(input)
          #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
          #print("Number of reviews processed: {}".format(i))
          input = "summarize: " + summaries_first_stage[i] + " . "
          count = len(summaries_first_stage[i].split(" "))

      if(count != 0):
        batches_second_stage.append(input)
        #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
        #print("Number of reviews processed: {}".format(i))

      summaries_second_stage = []
      for input in batches_second_stage:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      print("Product " + str(id) + ", year " + str(year) + ": " + str(summaries_second_stage))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-xsum')
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-xsum')

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-xsum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
summarize_final(df)